In [2]:
import pandas as pd
import numpy as np
import requests
import json

In [17]:
file_list = ["TRACKING_AVAILABLE.xlsx", "TRACKING_ONLINE.xlsx", "IN_US.xlsx", "COMPLETED.xlsx", "RETURN_TO_SENDER.xlsx" ]
li = []
for file in file_list:
    df = pd.read_excel(file)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
df.tail()

,id,tracking_number,update_time,lstm_status,bilstm_status,bert_status,tracking_available_date,tracking_online_date,in_us_date,delivered_guarantee_date,completed_date,return_to_sender_date
245,YT2101921236000752,YT2101921236000752,2021-02-02 12:00:00,RETURN_TO_SENDER,IN_US,RETURN_TO_SENDER,NaN,NaN,NaN,NaN,NaN,NaN
246,YT2100221272059417,YT2100221272059417,2021-01-19 14:08:00,RETURN_TO_SENDER,RETURN_TO_SENDER,RETURN_TO_SENDER,NaN,NaN,NaN,NaN,NaN,NaN
247,YT2036421272131660,YT2036421272131660,2021-01-19 14:37:00,RETURN_TO_SENDER,RETURN_TO_SENDER,RETURN_TO_SENDER,NaN,NaN,NaN,NaN,NaN,NaN
248,YT2101321272036069,YT2101321272036069,2021-02-24 05:13:57.161905,RETURN_TO_SENDER,NaN,RETURN_TO_SENDER,NaN,NaN,NaN,NaN,NaN,NaN
249,YT2101621236003420,YT2101621236003420,2021-02-03 06:10:00,RETURN_TO_SENDER,RETURN_TO_SENDER,RETURN_TO_SENDER,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df = df.drop(['tracking_available_date', 'tracking_online_date', 'in_us_date', 'delivered_guarantee_date', 'completed_date', 'return_to_sender_date'], axis=1)
df.tail()

,id,tracking_number,update_time,lstm_status,bilstm_status,bert_status
245,YT2101921236000752,YT2101921236000752,2021-02-02 12:00:00,RETURN_TO_SENDER,IN_US,RETURN_TO_SENDER
246,YT2100221272059417,YT2100221272059417,2021-01-19 14:08:00,RETURN_TO_SENDER,RETURN_TO_SENDER,RETURN_TO_SENDER
247,YT2036421272131660,YT2036421272131660,2021-01-19 14:37:00,RETURN_TO_SENDER,RETURN_TO_SENDER,RETURN_TO_SENDER
248,YT2101321272036069,YT2101321272036069,2021-02-24 05:13:57.161905,RETURN_TO_SENDER,NaN,RETURN_TO_SENDER
249,YT2101621236003420,YT2101621236003420,2021-02-03 06:10:00,RETURN_TO_SENDER,RETURN_TO_SENDER,RETURN_TO_SENDER


In [19]:
tracking_number_list = list(df['tracking_number'])
len(tracking_number_list)

250

In [38]:
tracking_number_list = [
"YT2012021272121165"


]

# Crawl data via api 17track

In [39]:
log_list = []

In [40]:
count = 0
for tracking_number in tracking_number_list:
    
    url = "https://api.17track.net/track/v1/gettrackinfo"

    payload="[{ \"number\": \""+ tracking_number +"\" }]"

    headers = {
      '17token': 'A36466A02D9B25A8C99C58CD5AC22414',
      'Content-Type': 'application/json'
    }
    
    response = requests.post(url, headers=headers, data=payload)

    if response.status_code == 200:
        obj = response.json()
        if obj['data']['accepted']:
            
            if obj['data']['accepted'][0]['track']['z1']:
                get_logs = obj['data']['accepted'][0]['track']['z1'][::-1]  
            elif obj['data']['accepted'][0]['track']['z2']:
                get_logs = obj['data']['accepted'][0]['track']['z2'][::-1]
                
            log_collection = ''

            for log in get_logs:
                log_collection += ( log['c'] + ' ' + log['d'] + ' ' + log['z'] + ' . ' )

                info = {'tracking_number': tracking_number, 'checkpoint_status': '', 'status_description': log_collection}

                log_list.append(info)

In [41]:
log_df = pd.DataFrame(log_list)
log_df.tail()

""


In [24]:
print("number of tracking_number: {}".format(len(set(list(log_df["tracking_number"])))))
print("number of logs: {}".format(log_df.shape[0]))

number of tracking_number: 35
number of logs: 912


In [25]:
log_df.to_excel("rts.xlsx", index=False)

In [14]:
tracking_number_df = pd.DataFrame(list(set(list(log_df["tracking_number"]))))
labels=["tracking_number"]
tracking_number_df.columns = labels
tracking_number_df.tail()

,tracking_number
30,YT2028721272024889
31,YT2028221272047603
32,YT2010821272127708
33,YT2007721272061526
34,YT2015321272076421


In [25]:
crawl_df = pd.merge(df, tracking_number_df, on="tracking_number")
crawl_df.tail()

,id,tracking_number,update_time,lstm_status,bilstm_status,bert_status
92,YT2101821272049309,YT2101821272049309,2021-02-03 10:56:00,RETURN_TO_SENDER,RETURN_TO_SENDER,RETURN_TO_SENDER
93,YT2101921236000561,YT2101921236000561,2021-02-03 08:58:00,RETURN_TO_SENDER,COMPLETED,RETURN_TO_SENDER
94,YT2101621236003689,YT2101621236003689,2021-02-03 12:05:00,RETURN_TO_SENDER,COMPLETED,COMPLETED
95,YT2101621236003569,YT2101621236003569,2021-02-05 15:25:00,RETURN_TO_SENDER,COMPLETED,RETURN_TO_SENDER
96,YT2036421272131660,YT2036421272131660,2021-01-19 14:37:00,RETURN_TO_SENDER,RETURN_TO_SENDER,RETURN_TO_SENDER


In [26]:
crawl_df.groupby("lstm_status").count()

,id,tracking_number,update_time,bilstm_status,bert_status
lstm_status,,,,,
COMPLETED,14,14,14,12,14
IN_US,17,17,17,11,17
RETURN_TO_SENDER,15,15,15,13,15
TRACKING_AVAILABLE,32,32,32,0,32
TRACKING_ONLINE,19,19,19,0,19


In [15]:
log_df.to_excel("test_data1.xlsx", index=False)

In [44]:
data = pd.read_csv("data_scm_tracking_ml.csv")
data.tail()

c:\users\admin\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,tracking_number,update_time,last_log_time,missing_order,lstm_status,bilstm_status,bert_status
74908,YT2107221236001439,YT2107221236001439,3/1/2021 18:24:16.570299,3/1/2021 18:24:16.570299,0,IN_US,NaN,IN_US
74909,YT2107521272078595,YT2107521272078595,3/1/2021 18:24:17.647138,3/1/2021 18:24:17.647138,0,TRACKING_ONLINE,NaN,TRACKING_ONLINE
74910,YT2107521272076718,YT2107521272076718,3/1/2021 18:24:08.826822,3/1/2021 18:24:08.826822,0,TRACKING_ONLINE,NaN,TRACKING_ONLINE
74911,YT2107521272080559,YT2107521272080559,3/1/2021 18:26:00.012744,3/1/2021 18:26:00.012744,0,TRACKING_ONLINE,NaN,TRACKING_ONLINE
74912,YT2106521236002562,YT2106521236002562,30/3/2021 21:22:18.907399,19/3/2021 17:02:00,0,COMPLETED,NaN,COMPLETED


In [45]:
data.shape

(74913, 8)

In [46]:
data.dtypes

id                 object
tracking_number    object
update_time        object
last_log_time      object
missing_order       int64
lstm_status        object
bilstm_status      object
bert_status        object
dtype: object

In [47]:
import random
random.randint(0, 1)

1

In [54]:
def my_rand(a):
    return random.randint(0, 1)

my_rand(0)

1

In [55]:
data["missing_order"] = data["missing_order"].apply(my_rand)

In [56]:
data.head()

,id,tracking_number,update_time,last_log_time,missing_order,lstm_status,bilstm_status,bert_status
0,YT2100921272131268,YT2100921272131268,29/1/2021 14:14:00,29/1/2021 14:14:00,0,COMPLETED,COMPLETED,RETURN_TO_SENDER
1,YT2101621236004007,YT2101621236004007,30/1/2021 13:54:00,30/1/2021 13:54:00,0,COMPLETED,COMPLETED,COMPLETED
2,YT2103221272032772,YT2103221272032772,2/1/2021 21:26:01.553249,2/1/2021 21:26:01.553249,0,COMPLETED,NaN,COMPLETED
3,YT2101921236003761,YT2101921236003761,2/2/2021 16:02:00,2/2/2021 16:02:00,1,COMPLETED,COMPLETED,RETURN_TO_SENDER
4,YT2101221236003755,YT2101221236003755,3/2/2021 09:59:00,3/2/2021 09:59:00,0,COMPLETED,COMPLETED,COMPLETED


In [57]:
data.groupby("missing_order").count()

,id,tracking_number,update_time,last_log_time,lstm_status,bilstm_status,bert_status
missing_order,,,,,,,
0,37466,37466,37466,37466,37466,3313,37466
1,37447,37447,37447,37447,37447,3171,37447


In [ ]:
data.to_csv("")